The aim of this exercise is to analyse statistics of a hot wire experiment dataset.

The first step is to visualise the voltage time series. Then we build a calibration function to obtain the resulting velocity time series. We may then perform statistical analysis to evaluate statistical convergence.

I am importing the necessary python packages first

In [1]:
# Importing packages
import os # Required to navigate files/folders
import numpy as np # Required for handling of matrices and statistical operations.
import matplotlib.pyplot as plt # Plotting library
import matplotlib.ticker as ticker # Tools for tick placement
import seaborn as sns # Optional. I like to use seaborn colour palettes

I like to specify fonts for my plots right at the beginning. So I'm going to do that:

In [2]:
# Plot settings
plt.rcParams.update({'font.size' : 18, 'font.family' : 'serif', "text.usetex": True}) # I have chosen my font family and I have enabled the LaTeX interpreter.
colours=sns.color_palette(palette='Set1')
dpi = 100

In [3]:
# Timeseries import
signal = np.loadtxt('Timeseries.txt', delimiter=None, converters=None, skiprows=0, usecols=None, unpack=False, ndmin=0, encoding=None, max_rows=None)

I now create the time vector on the basis of the acquisition frequency and sample count

In [4]:
# Acquisition parameters
num_samples = len(signal)
sampling_freq = 10000
dt = 1/sampling_freq

# Time vector definition
time = np.linspace(0,(num_samples-1)*dt,num_samples) # Linspace (start, stop, number of samples)


I can now make a line plot to visualise our signal (ugly graph, but we'll worry about aesthetics when we get to plotting velocity).

In [ ]:
# Plot to visualise timeseries
max_x = 5*(np.floor_divide(np.max(time),5)+1)
fig1, ax = plt.subplots(figsize=(10, 4), dpi=dpi)
ax.plot(time, signal, alpha = 1, label='Voltage timeseries', linewidth=0.5, color=colours[1])
ax.set_xlim([0,max_x])
ax.set_ylim([np.min(signal)-0.0005,np.max(signal)+0.0005])
ax.set_xticks(np.arange(0,max_x,5))
ax.set_yticks(np.arange(np.min(signal),np.max(signal),0.005))
ax.set_ylabel(r'Voltage (V)', fontsize=16) 
ax.set_xlabel(r'Time (s)',fontsize=16)  # label the y axis
plt.grid(True)
fig1.savefig('voltage_timeseries.png', dpi=600,bbox_inches="tight")
plt.show() 


We need to convert this voltage timeseries to a velocity timeseries.

Let us write a calibration function to extract the calibration coefficients. In this example, I will fit the calibration data to a 3rd order polynomial. 

You are encouraged to try different polynomial orders and functions (King's law, modified King's Law, etc.)

In your final report, include an analysis of the goodness of fit.

In [ ]:
# Define calibration points (stencil)
cal_vel = [0,0.87,1.98,3.43,5.49,8.21,10.42,16.44] # Velocity list in m/s
order = 3 # Order of polynomial function for calibration

def calibrate_poly(cal_vel, order):
    volt_avg = []  # Initialise voltage array
    for vel in cal_vel:
        buffer = np.loadtxt('Calib/'+str(vel)+'.txt')  # Load the voltage timeseires for each velocity 
        volt_avg.append(np.abs(np.mean(buffer)))               # Calculate time-averaged voltage value

    coeffs=np.polyfit(volt_avg,cal_vel,order)             # Fit to a polynomial of order "order"
    print('Calibration function is:'+str(coeffs[0])+'x^3 + '+str(coeffs[1])+'x^2 + '+str(coeffs[2])+'x + '+str(coeffs[3]))
    return(coeffs, volt_avg)

calib, volts = calibrate_poly(cal_vel, order)  # Perform a calibration fitting using the defined calibration function

Let's plot the resulting calibration curve to evaluate the fit qualitatively:

In [ ]:
# Initialise a velocity vector to visualise calibration
voltsy=np.linspace(0,np.ceil(np.max(volts)/0.5)*0.5, 1000)
velx= np.polyval(calib, voltsy)

# Plot the calibration curve and calibration points
fig2, ax = plt.subplots(figsize=(6, 6), dpi=dpi)
ax.scatter(cal_vel, np.abs(volts), s=50, alpha=0.7, color=colours[1], edgecolors= "black")
ax.plot(velx, voltsy, alpha=0.8, linewidth=2, color="black")
ax.set_xlim([0,np.max(velx)])
ax.set_ylim([np.min(volts),np.max(voltsy)])
ax.set_xlabel(r'Velocity (m/s)', fontsize=16) 
ax.set_ylabel(r'Voltage $E$ (V)',fontsize=16)  
ax.set_title(r'Calibration Curve',fontsize=18)  
ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
ax.xaxis.set_major_formatter(ticker.FormatStrFormatter("%.0f"))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))
ax.xaxis.set_minor_formatter(plt.NullFormatter())
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%.2f"))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.1))
ax.yaxis.set_minor_formatter(plt.NullFormatter())
ax.tick_params(axis="both", which="major")
plt.grid(True,  which='major')
plt.grid(True,  which='minor', linestyle='--', alpha=0.5)
fig2.savefig('calibration.png', dpi=600,bbox_inches="tight")
plt.show() 

What do you think of the obtained fit? How does it behave at low velocities where data points are sparse? How it does behave when extrapolated to velocities higher than the highest calibrated value? Do you see signs of over-fitting?

Exercise: Try fitting polynomials of various orders. Try fitting a physical model - King's Law/ Modified King's Law.

Let's have a quantitative estimate of the fitting operation by analysing the mean squared error:

In [ ]:
# Evaluate the calibration function at the calibration points
model_vel = np.polyval(calib,volts)
# Evaluate the Mean Square Error (MSE)
MSE = np.square(np.subtract(cal_vel,model_vel)).mean()
print("Mean Square Error (MSE) = "+str(MSE)+" (m/s)^2")
print("This corresponds to an average velocity estimation error of "+str(np.round(np.sqrt(MSE),3))+" m/s due to the fitting")

Is this error acceptable? How does it compare to the velocity magnitudes for the intended measurement? How does this error compare to the error in obtaining the calibration velocities in the first place (error in velocity estimation from the Prandtl probe)?

Now that we have a set of calibration coefficients, we can convert our original acquired voltage timeseries into a velocity timeseries.

Note: If the voltage timeseries has been subjected to analog pre-amplification, we need to convert the amplified vvalues to calibration levels.

In [ ]:
# Define amplification parameters 
gain = 10
offset = 4

# Convert the timeseries to calibration form
signal=gain*(signal-offset)

# Apply the calibration
U=np.polyval(calib,np.abs(signal))

# Plot the velocity timeseries
max_x = 5*(np.floor_divide(np.max(time),5)+1)
fig3, ax = plt.subplots(figsize=(10, 4), dpi=dpi)
ax.plot(time, U, alpha = 1, label='Voltage timeseries', linewidth=0.5, color=colours[1])
ax.set_xlim([0,max_x])
ax.set_ylim([np.min(U),np.max(U)])
ax.set_xticks(np.arange(0,max_x,5))
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.025))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%.3f"))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.01))
ax.yaxis.set_minor_formatter(plt.NullFormatter())
ax.set_ylabel(r'$U$ (m/s)', fontsize=16) 
ax.set_xlabel(r'Time (s)',fontsize=16)  # label the y axis
plt.grid(True)
fig3.savefig('velocity_timeseries.png', dpi=600,bbox_inches="tight")
plt.show() 